# Enterprise Search Assistant using RAG + Agentic AI + LangChain

Install necessary packages

In [1]:
!pip install langchain openai faiss-cpu python-dotenv tiktoken

   ---------------------------------------- 0.0/680.4 kB ? eta -:--:--
   ---------------------------------------- 680.4/680.4 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.0 MB ? eta -:--:--
   -------- ------------------------------- 3.1/15.0 MB 14.2 MB/s eta 0:00:01
   ---------------- ----------------------- 6.3/15.0 MB 14.9 MB/s eta 0:00:01
   ------------------------ --------------- 9.2/15.0 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------  14.9/15.0 MB 17.7 MB/s eta 0:00:01
   ---------------------------------------- 15.0/15.0 MB 17.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   --------------------------------------- 894.9/894.9 kB 42.2 MB/s eta 0:00:00


In [3]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory

In [4]:
# Load API keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Load and Split Company Knowledge Files
def load_and_chunk_knowledge(file_path):
    loader = TextLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return splitter.split_documents(documents)

# Step 2: Create Vector Index
def create_vector_store(chunks):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(chunks, embeddings)
    return vector_store

# Step 3: Build Retrieval QA Chain
def build_qa_chain(vector_store):
    retriever = vector_store.as_retriever(search_type="similarity", k=5)
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")
    return qa_chain

# Step 4: Define Tools for the Agent
def define_tools(qa_chain):
    return [
        Tool(
            name="CompanyKnowledgeSearch",
            func=qa_chain.run,
            description="Useful for answering questions about company policies, procedures, and documentation"
        )
    ]

# Step 5: Initialize Agent
def create_agent(tools):
    llm = ChatOpenAI(model_name="gpt-4")
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    agent = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)
    return agent

# Step 6: Run Assistant Loop
def run_assistant():
    print("\n🔍 Enterprise Search Assistant Ready. Ask your company-related questions below.")
    print("Type 'exit' to quit.\n")

    # Load and build
    chunks = load_and_chunk_knowledge("company_docs/company_wiki.txt")
    vector_store = create_vector_store(chunks)
    qa_chain = build_qa_chain(vector_store)
    tools = define_tools(qa_chain)
    agent = create_agent(tools)

    while True:
        query = input("You: ")
        if query.lower() in ["exit", "quit"]:
            print("Assistant: Goodbye!")
            break
        response = agent.run(query)
        print(f"Assistant: {response}\n")

if __name__ == "__main__":
    run_assistant()



🔍 Enterprise Search Assistant Ready. Ask your company-related questions below.
Type 'exit' to quit.



RuntimeError: Error loading company_docs/company_wiki.txt